In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt


from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel, NBEATSModel, LinearRegressionModel
from darts.models.forecasting.xgboost import XGBModel
from darts.models.forecasting.rnn_model import RNNModel

from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, smape, mae, rmse
from darts.utils.utils import ModelMode

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [3]:

data = pd.read_csv('Cleaned HIV Data.csv')

In [6]:
cutoff_year = "2017"

In [11]:
series = TimeSeries.from_dataframe(data,
                                    time_col="Time"
                                    )

train_series, val_series = series.split_before(pd.Timestamp(cutoff_year))
# scaler = Scaler()
# training_scaled = scaler.fit_transform(train_series)
# val_scaled = scaler.transform(val_series)


In [32]:
total_countries = len(data.columns)-1

In [20]:
es_result = 0

for country in data.columns:

    if country != "Time":
        scaler = Scaler()
        training_scaled = scaler.fit_transform(train_series[country])
        val_scaled = scaler.transform(val_series[country])

        es_model = ExponentialSmoothing(trend=ModelMode.ADDITIVE, seasonal=None)
        es_model.fit(training_scaled)
        pred_new_cases = es_model.predict(len(val_scaled))
        pred = rmse(pred_new_cases, val_scaled)
        es_result+=pred

In [33]:
print(f"The average RMSE of each country using Exponential Smoothing is {es_result/total_countries}")

The average RMSE of each country using Exponential Smoothing is 0.15353690708650983


In [28]:
lr_result = 0

for country in data.columns:

    if country != "Time":
        scaler = Scaler()
        training_scaled = scaler.fit_transform(train_series[country])
        val_scaled = scaler.transform(val_series[country])

        lr = LinearRegressionModel(lags=1)
        lr.fit(train_series["South Africa"])
        pred_new_cases = es_model.predict(len(val_scaled))
        pred = rmse(pred_new_cases, val_scaled)
        lr_result+=pred

In [34]:
print(f"The average RMSE of each country using Exponential Smoothing is {lr_result/total_countries}")

The average RMSE of each country using Exponential Smoothing is 0.36129435108022334


In [29]:
lr_result

16.619540149690273

46